In [1]:
import os
directory = os.getcwd()  
directory = directory.split('src')
path = directory[0]
%cd $path
%pwd

c:\Users\JENMOSQU\Documents\PERSONAL\cod\Probabilistic-Electricity-Demand-


'c:\\Users\\JENMOSQU\\Documents\\PERSONAL\\cod\\Probabilistic-Electricity-Demand-'

In [2]:
# Libraries..

import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from itertools import product
from tqdm import tqdm_notebook
import pandas as pd
import copy

# Plotly and Matplotlib
import plotly.express as px
import plotly.graph_objects as go
import plotly.express as px
# import matplotlib.pyplot as plt
# import matplotlib.ticker as ticker


# sklearn
from sklearn.compose import make_column_transformer

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error,mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import KNNImputer

# Keras

from keras.models import Sequential
from keras.backend import clear_session
from keras.callbacks import EarlyStopping
from keras_tuner.tuners import RandomSearch
from keras.layers import SimpleRNN, LSTM, GRU, Dropout, Dense, Bidirectional

# tensorflow
from tensorflow.keras.models import load_model

# Modules from repo
from src.Utilities import utilities as tool



# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')
# Configuración options.display
# ==============================================================================
pd.set_option('display.max_rows', 5000)
pd.options.display.max_rows = None
pd.options.display.max_columns = 5000
pd.options.display.max_colwidth = 5000
pd.options.display.max_dir_items = 5000
pd.options.display.width = 5000
pd.options.display.float_format = '{:.1f}'.format



Using TensorFlow backend


In [3]:
#Import Data..............

df=pd.read_parquet('df.parquet')
columnas_indeseadas =['HR_CAL_MEDIA_D', 'VVAG_MEDIA_D'] #columna con correlación 0 con la demanda
columnas_indeseadas += [f'Demanda_SIN_{i}' for i in range(1, 29)]
# columnas_indeseadas += [f'Demanda_SIN_{i}' for i in range(7, 29, 7)]
print("columnas rezagadas que no son necesarias porque skforecast rezaga internamente",columnas_indeseadas)
df = df.loc[:, ~df.columns.isin(columnas_indeseadas)]
print(df.head(1))
print("--------------------------------------------------------------------------------------------------")
print(df.tail(1))

columnas rezagadas que no son necesarias porque skforecast rezaga internamente ['HR_CAL_MEDIA_D', 'VVAG_MEDIA_D', 'Demanda_SIN_1', 'Demanda_SIN_2', 'Demanda_SIN_3', 'Demanda_SIN_4', 'Demanda_SIN_5', 'Demanda_SIN_6', 'Demanda_SIN_7', 'Demanda_SIN_8', 'Demanda_SIN_9', 'Demanda_SIN_10', 'Demanda_SIN_11', 'Demanda_SIN_12', 'Demanda_SIN_13', 'Demanda_SIN_14', 'Demanda_SIN_15', 'Demanda_SIN_16', 'Demanda_SIN_17', 'Demanda_SIN_18', 'Demanda_SIN_19', 'Demanda_SIN_20', 'Demanda_SIN_21', 'Demanda_SIN_22', 'Demanda_SIN_23', 'Demanda_SIN_24', 'Demanda_SIN_25', 'Demanda_SIN_26', 'Demanda_SIN_27', 'Demanda_SIN_28']
       FECHA TIPO_DIA  Demanda_SIN  Covid  Cambio_nivel  Tiempo  precipitacion  Temperatura  HRA2_MN_D  HRA2_MX_D  THSM_MEDIA_D  THSM_MX_D  VV_10_MEDIA_D  VV_10_VECT_MEDIA_D  THSM_MN_D  PT_2_TT_D  PT_2_MX_D  PT_10_MX_D  PT_10_TT_D  TSSM_MEDIA_D  TSSM_MN_D  TSSM_MX_D  PTPG_TT_D  BSHG_TT_D  TPR_CAL
0 2017-12-01  VIERNES     193865.8      0             0       1            9.6         22.2  

In [4]:
#Correlación variables y filtrado........
#['THSM_MEDIA_D','PTPG_TT_D']
correlacion_con_demanda = df.corr()['Demanda_SIN']
nombres_variables = correlacion_con_demanda[(correlacion_con_demanda >= 0.2) | (correlacion_con_demanda <= -0.2)]
print(nombres_variables)
nombres_variables=nombres_variables.index.tolist()
nombres_variables +=['THSM_MEDIA_D'] #'PTPG_TT_D' #'VVAG_MEDIA_D'
nombres_variables.remove('Demanda_SIN')
df=df[['FECHA', 'TIPO_DIA', 'Demanda_SIN'] + nombres_variables]
print("columnas finales", df.columns)
df.head(2)

Demanda_SIN           1.0
Covid                -0.2
Cambio_nivel          0.4
Tiempo                0.6
VV_10_MEDIA_D         0.2
VV_10_VECT_MEDIA_D    0.2
PT_10_TT_D           -0.3
Name: Demanda_SIN, dtype: float64
columnas finales Index(['FECHA', 'TIPO_DIA', 'Demanda_SIN', 'Covid', 'Cambio_nivel', 'Tiempo', 'VV_10_MEDIA_D', 'VV_10_VECT_MEDIA_D', 'PT_10_TT_D', 'THSM_MEDIA_D'], dtype='object')


,FECHA,TIPO_DIA,Demanda_SIN,Covid,Cambio_nivel,Tiempo,VV_10_MEDIA_D,VV_10_VECT_MEDIA_D,PT_10_TT_D,THSM_MEDIA_D
0,2017-12-01,VIERNES,193865.8,0,0,1,1.7,0.9,6.2,19.1
1,2017-12-02,SABADO,184393.8,0,0,2,1.8,0.9,7.9,19.2


In [5]:
#Imputation: Rolling mean........................

columns_rolling = ['VV_10_MEDIA_D', 'VV_10_VECT_MEDIA_D','PT_10_TT_D' ,'THSM_MEDIA_D']
window_size=60
df = tool.exploratory_tools.fill_null_values_with_rolling_mean(df, columns_rolling, window_size)


In [6]:
# Transformer one hot encoding and formatting-.......

print("--------------------------------------------------------------------------------------------------------------------------------------")
# ==============================================================================
one_hot_encoder = make_column_transformer(
                      #(StandardScaler(), ['Demanda_SIN']),
                      (   
                          OneHotEncoder(sparse_output=False, drop='if_binary'),
                          ['TIPO_DIA'],
                      ),
                      remainder="passthrough",
                      verbose_feature_names_out=False,
                  ).set_output(transform="pandas")

df = one_hot_encoder.fit_transform(df)

#-------------------------------------------------------------------------------------------------------------------------------------------
#Variable TIPO_DIA
#-------------------------------------------------------------------------------------------------------------------------------------------
columnas_tipo_dia = [col for col in df.columns if col.startswith('TIPO_DIA_')]
variables_exogenas = ['FECHA'] + nombres_variables + columnas_tipo_dia
variable_dependiente = ['Demanda_SIN']
df = df[variable_dependiente + variables_exogenas] 

#-------------------------------------------------------------------------------------------------------------------------------------------
# Codificación de Fecha...
#-------------------------------------------------------------------------------------------------------------------------------------------
df['FECHA'] = pd.to_datetime(df['FECHA'], format='%Y-%m-%d')
df = df.set_index('FECHA')
df = df.asfreq('D')


df.head(2)

--------------------------------------------------------------------------------------------------------------------------------------


,Demanda_SIN,Covid,Cambio_nivel,Tiempo,VV_10_MEDIA_D,VV_10_VECT_MEDIA_D,PT_10_TT_D,THSM_MEDIA_D,TIPO_DIA_1--ENE,TIPO_DIA_1--MAY,TIPO_DIA_2--ENE,TIPO_DIA_20--JUL,TIPO_DIA_24--DIC,TIPO_DIA_25--DIC,TIPO_DIA_31--DIC,TIPO_DIA_7--AGO,TIPO_DIA_8--DIC,TIPO_DIA_DOALF,TIPO_DIA_DOALFENE,TIPO_DIA_DOMINGO,TIPO_DIA_DOVDIC,TIPO_DIA_DOVENE,TIPO_DIA_DSS,TIPO_DIA_JSS,TIPO_DIA_JUEVES,TIPO_DIA_JUVDIC,TIPO_DIA_JUVENE,TIPO_DIA_LF,TIPO_DIA_LFENE,TIPO_DIA_LUNES,TIPO_DIA_LUVDIC,TIPO_DIA_LUVENE,TIPO_DIA_MADLF,TIPO_DIA_MARTES,TIPO_DIA_MAVDIC,TIPO_DIA_MAVENE,TIPO_DIA_MIERCOLES,TIPO_DIA_MIVDIC,TIPO_DIA_MIVENE,TIPO_DIA_MSS,TIPO_DIA_SAALF,TIPO_DIA_SAALFENE,TIPO_DIA_SABADO,TIPO_DIA_SAVDIC,TIPO_DIA_SAVENE,TIPO_DIA_SSS,TIPO_DIA_VIERNES,TIPO_DIA_VIVDIC,TIPO_DIA_VIVENE,TIPO_DIA_VSS
FECHA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-12-01,193865.8,0,0,1,1.7,0.9,6.2,19.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2017-12-02,184393.8,0,0,2,1.8,0.9,7.9,19.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# Separación datos train-val-test................
#-------------------------------------------------------------------------------------------------------------------------------------------

datos = df[df.index >= '2017-12-01']
fin_train = '2023-05-31'
fin_validacion = '2023-09-23'

datos_train = datos[datos.index <= fin_train].copy()
datos_val = datos[(datos.index > fin_train) & (datos.index <= fin_validacion)].copy()
datos_test = datos[datos.index > fin_validacion].copy()

print(f"Fechas train      : {datos_train.index.min()} --- {datos_train.index.max()}  (n={len(datos_train)})")
print(f"Fechas validacion : {datos_val.index.min()} --- {datos_val.index.max()}  (n={len(datos_val)})")
print(f"Fechas test       : {datos_test.index.min()} --- {datos_test.index.max()}  (n={len(datos_test)})")
print("-------------------------------------------------------------------------------------------------------------------------------------")

datos_test.head(2)

Fechas train      : 2017-12-01 00:00:00 --- 2023-05-31 00:00:00  (n=2008)
Fechas validacion : 2023-06-01 00:00:00 --- 2023-09-23 00:00:00  (n=115)
Fechas test       : 2023-09-24 00:00:00 --- 2023-09-30 00:00:00  (n=7)
-------------------------------------------------------------------------------------------------------------------------------------


,Demanda_SIN,Covid,Cambio_nivel,Tiempo,VV_10_MEDIA_D,VV_10_VECT_MEDIA_D,PT_10_TT_D,THSM_MEDIA_D,TIPO_DIA_1--ENE,TIPO_DIA_1--MAY,TIPO_DIA_2--ENE,TIPO_DIA_20--JUL,TIPO_DIA_24--DIC,TIPO_DIA_25--DIC,TIPO_DIA_31--DIC,TIPO_DIA_7--AGO,TIPO_DIA_8--DIC,TIPO_DIA_DOALF,TIPO_DIA_DOALFENE,TIPO_DIA_DOMINGO,TIPO_DIA_DOVDIC,TIPO_DIA_DOVENE,TIPO_DIA_DSS,TIPO_DIA_JSS,TIPO_DIA_JUEVES,TIPO_DIA_JUVDIC,TIPO_DIA_JUVENE,TIPO_DIA_LF,TIPO_DIA_LFENE,TIPO_DIA_LUNES,TIPO_DIA_LUVDIC,TIPO_DIA_LUVENE,TIPO_DIA_MADLF,TIPO_DIA_MARTES,TIPO_DIA_MAVDIC,TIPO_DIA_MAVENE,TIPO_DIA_MIERCOLES,TIPO_DIA_MIVDIC,TIPO_DIA_MIVENE,TIPO_DIA_MSS,TIPO_DIA_SAALF,TIPO_DIA_SAALFENE,TIPO_DIA_SABADO,TIPO_DIA_SAVDIC,TIPO_DIA_SAVENE,TIPO_DIA_SSS,TIPO_DIA_VIERNES,TIPO_DIA_VIVDIC,TIPO_DIA_VIVENE,TIPO_DIA_VSS
FECHA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-09-24,203949.3,0,1,2124,49.4,49.9,5.8,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-09-25,233453.1,0,1,2125,49.4,50.0,5.9,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
#Split and reshape.....

#DATOS DE ENTRENAMIENTO
X_train = datos_train.drop("Demanda_SIN", axis=1)
y_train = datos_train["Demanda_SIN"]
X_train = X_train.values
X_train = X_train.reshape((X_train.shape[0],1,  X_train.shape[1]))
print(X_train.shape)

# DATOS DE VALIDACIÓN
X_val = datos_val.drop("Demanda_SIN", axis=1)
y_val = datos_val["Demanda_SIN"]
X_val=X_val.values
X_val = X_val.reshape((X_val.shape[0],1,  X_val.shape[1]))
print(X_val.shape)

# DATOS DE TEST
X_test = datos_test.drop("Demanda_SIN", axis=1)
y_test = datos_test["Demanda_SIN"]
X_test=X_test.values
X_test = X_test.reshape((X_test.shape[0],1,  X_test.shape[1]))
print(X_test.shape)

(2008, 1, 49)
(115, 1, 49)
(7, 1, 49)


In [9]:
early_stopping = EarlyStopping(monitor="val_loss", patience=20)
early_stopping

### Red neuronal recurrente (RNN) simple

In [10]:
rnn_tuner = RandomSearch(hypermodel=tool.rnn_tools.create_simple_rnn,
                         objective="val_mse",
                         max_trials=10,
                         seed=42,
                         directory="RNN")

rnn_tuner.search(X_train, y_train, epochs=1000, validation_data=(X_val, y_val), callbacks=[early_stopping]) 

Reloading Tuner from RNN\untitled_project\tuner0.json


In [11]:
rnn = rnn_tuner.get_best_models(num_models=1)[0]
rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 1, 96)             14016     
                                                                 
 dropout (Dropout)           (None, 1, 96)             0         
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 1, 128)            28800     
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 1, 32)             5152      
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 96)                12384     
                                                                 
 dropout_1 (Dropout)         (None, 96)                0         
                                                                 
 dense (Dense)               (None, 1)                 9

In [12]:
#Save model...
#-----------------------------------------------------------------------------------------------------------
rnn.save("Rnn_models/rnn.h5")
#-----------------------------------------------------------------------------------------------------------

print(f"Número de parámetros entrenables: {rnn.count_params()}")

for i, activation in enumerate([layer.activation for layer in rnn.layers if hasattr(layer, 'activation')]):
    print(f'Capa {i}: Función de activación = {activation}')

Número de parámetros entrenables: 60449
Capa 0: Función de activación = <function relu at 0x000001B2A4F20EE0>
Capa 1: Función de activación = <function selu at 0x000001B2A4F20820>
Capa 2: Función de activación = <function relu at 0x000001B2A4F20EE0>
Capa 3: Función de activación = <function relu at 0x000001B2A4F20EE0>
Capa 4: Función de activación = <function linear at 0x000001B2A4F2A940>


### LSTM

In [13]:
clear_session()

lstm_tuner = RandomSearch(hypermodel=tool.rnn_tools.create_lstm, 
                          objective="val_mse",
                          max_trials=10,
                          seed=42,
                          directory="LSTM")

lstm_tuner.search(X_train, y_train, epochs=1000, validation_data=(X_val, y_val), callbacks=[early_stopping])

Trial 10 Complete [00h 00m 29s]
val_mse: 210568144.0

Best val_mse So Far: 74764848.0
Total elapsed time: 00h 50m 44s


In [14]:
lstm = lstm_tuner.get_best_models(num_models=1)[0]
lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 96)             56064     
                                                                 
 dropout (Dropout)           (None, 1, 96)             0         
                                                                 
 lstm_1 (LSTM)               (None, 1, 128)            115200    
                                                                 
 lstm_2 (LSTM)               (None, 1, 32)             20608     
                                                                 
 lstm_3 (LSTM)               (None, 32)                8320      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 3

In [15]:
#Save model...
#-----------------------------------------------------------------------------------------------------------
lstm.save("Rnn_models/lstm.h5")
#-----------------------------------------------------------------------------------------------------------

for i, activation in enumerate([layer.activation for layer in lstm.layers if hasattr(layer, 'activation')]):
    print(f'Capa {i}: Función de activación = {activation}')

Capa 0: Función de activación = <function relu at 0x000001B2A4F20EE0>
Capa 1: Función de activación = <function relu at 0x000001B2A4F20EE0>
Capa 2: Función de activación = <function relu at 0x000001B2A4F20EE0>
Capa 3: Función de activación = <function relu at 0x000001B2A4F20EE0>
Capa 4: Función de activación = <function linear at 0x000001B2A4F2A940>


### GRU

In [16]:
gru_tuner = RandomSearch(hypermodel=tool.rnn_tools.create_gru, 
                         objective="val_mse",
                         max_trials=10,
                         seed=42,
                         directory="GRU")

gru_tuner.search(X_train, y_train, epochs=1000, validation_data=(X_val, y_val), callbacks=[early_stopping])

Trial 10 Complete [00h 00m 44s]
val_mse: 232235184.0

Best val_mse So Far: 61491932.0
Total elapsed time: 00h 49m 31s


In [17]:
gru = gru_tuner.get_best_models(num_models=1)[0]
gru.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 1, 96)             42336     
                                                                 
 dropout (Dropout)           (None, 1, 96)             0         
                                                                 
 gru_1 (GRU)                 (None, 1, 128)            86784     
                                                                 
 gru_2 (GRU)                 (None, 1, 32)             15552     
                                                                 
 gru_3 (GRU)                 (None, 32)                6336      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 3

In [18]:
#Save model...
#-----------------------------------------------------------------------------------------------------------
gru.save("Rnn_models/gru.h5")
#-----------------------------------------------------------------------------------------------------------

# Obtener las capas de activación para cada capa de la red
activation_layers = [layer.activation for layer in gru.layers if hasattr(layer, 'activation')]

# Imprimir las funciones de activación elegidas para cada capa
for i, activation in enumerate(activation_layers):
    print(f'Capa {i}: Función de activación = {activation}')

Capa 0: Función de activación = <function relu at 0x000001B2A4F20EE0>
Capa 1: Función de activación = <function relu at 0x000001B2A4F20EE0>
Capa 2: Función de activación = <function relu at 0x000001B2A4F20EE0>
Capa 3: Función de activación = <function relu at 0x000001B2A4F20EE0>
Capa 4: Función de activación = <function linear at 0x000001B2A4F2A940>


### Bidirectional LSTM

In [21]:
clear_session()

bilstm_tuner = RandomSearch(hypermodel=tool.rnn_tools.create_bidirectionallstm, 
                          objective="val_mse",
                          max_trials=10,
                          seed=42,
                          directory="BiLSTM")

bilstm_tuner.search(X_train, y_train, epochs=1000, validation_data=(X_val, y_val), callbacks=[early_stopping])

Trial 10 Complete [00h 00m 56s]
val_mse: 151661488.0

Best val_mse So Far: 68553184.0
Total elapsed time: 01h 36m 00s


In [22]:
best_hyperparameters = bilstm_tuner.get_best_hyperparameters(num_trials=1)[0]
bidirectional_lstm = bilstm_tuner.get_best_models(num_models=1)[0]

bidirectional_lstm.build((None, 1, 49)) 

bidirectional_lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirection  (None, 1, 192)            112128    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 1, 192)            0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 1, 256)            328704    
 onal)                                                           
                                                                 
 bidirectional_2 (Bidirecti  (None, 1, 64)             73984     
 onal)                                                           
                                                                 
 bidirectional_3 (Bidirecti  (None, 64)                24832     
 onal)                                                  

In [23]:
#Save model...
#-----------------------------------------------------------------------------------------------------------
bidirectional_lstm.save("Rnn_models/bidirectional_lstm.h5")
#-----------------------------------------------------------------------------------------------------------

### EVALUACIÓN

In [11]:
rnn  = load_model("Rnn_models/rnn.h5")
lstm = load_model("Rnn_models/lstm.h5")
gru = load_model("Rnn_models/gru.h5")
bidirectional_lstm = load_model("Rnn_models/bidirectional_lstm.h5")

In [12]:
models = [("RNN", rnn), ("LSTM", lstm),  ("GRU", gru), ("BidirectionalLSTM", bidirectional_lstm)] 
 
for name, model in models:
    print(f"Las metricas para la arquitectura {name}:")
    evaluation = tool.RegressionEvaluator(predicted=model.predict(X_test, verbose=0).ravel(), 
                                     observed=y_test)
    evaluation.print_metrics()
    print(f"--" * 30)

Las metricas para la arquitectura RNN:
El RMSE es: 7634.6477
El MAE es: 6812.3114
El MAPE es: 0.0302
------------------------------------------------------------
Las metricas para la arquitectura LSTM:
El RMSE es: 7114.5514
El MAE es: 6658.4878
El MAPE es: 0.0293
------------------------------------------------------------
Las metricas para la arquitectura GRU:
El RMSE es: 7352.4034
El MAE es: 7203.0614
El MAPE es: 0.0311
------------------------------------------------------------
Las metricas para la arquitectura BidirectionalLSTM:
El RMSE es: 7300.0224
El MAE es: 7033.5034
El MAPE es: 0.0308
------------------------------------------------------------


In [22]:
0.0308*100

3.08

In [13]:
test = pd.concat(objs=[pd.Series(model.predict(X_test, verbose=0).ravel(),
                                 name=f"{name}_forecast",
                                 index=y_test.index) for name, model in models], axis=1).join(y_test)
test.head(2)
test.rename(columns={'Demanda_SIN': 'Real Demand'}, inplace=True)

In [32]:
import plotly.express as px

# Colores utilizados en el gráfico anterior
color_discrete_sequence = ['#276981', '#9e9764', '#642424', '#001a57', '#e59882']

# Crear un gráfico de líneas
fig = px.line(test, x=test.index, y=['RNN_forecast', 'LSTM_forecast', 'GRU_forecast', 'BidirectionalLSTM_forecast', 'Real Demand'],
              color_discrete_sequence=color_discrete_sequence)

# Definir un conjunto de estilos de líneas (line_dash) para cada línea
line_dash_styles = ['solid', 'dash', 'dot', 'dashdot', 'longdash']

# Asignar un estilo de línea diferente a cada traza en el gráfico
for i, trace in enumerate(fig.data):
    trace.line.dash = line_dash_styles[i % len(line_dash_styles)]

# Personalizar el gráfico
fig.update_layout(
    xaxis_title="Date-daily",
    yaxis_title="Electricity Demand SIN kWh",
)

# Mostrar el gráfico
fig.show()


In [35]:
pronostico_xm=['198935.479304','225441.965435','231708.272221','232350.914002','232588.136322','233661.526792','220663.551312']
pronostico_xm = [float(value) for value in pronostico_xm]

# Agregar la nueva columna al DataFrame
test['XM Forecast'] = pronostico_xm
#test.reset_index()

In [37]:
import plotly.express as px

# Colores utilizados en el gráfico anterior
color_discrete_sequence = ['#276981', '#9e9764', '#642424', '#001a57', '#e59882', '#9e9764']

# Crear un gráfico de líneas
fig = px.line(test, x=test.index, y=['RNN_forecast', 'LSTM_forecast', 'GRU_forecast', 'BidirectionalLSTM_forecast', 'Real Demand', 'XM Forecast'],
              color_discrete_sequence=color_discrete_sequence)

# Definir un conjunto de estilos de líneas (line_dash) para cada línea
line_dash_styles = ['solid', 'dash', 'dot', 'dashdot', 'longdash']

# Asignar un estilo de línea diferente a cada traza en el gráfico
for i, trace in enumerate(fig.data):
    trace.line.dash = line_dash_styles[i % len(line_dash_styles)]

# Personalizar el gráfico
fig.update_layout(
    xaxis_title="Date-daily",
    yaxis_title="Electricity Demand SIN kWh",
)

# Mostrar el gráfico
fig.show()

### XM_pronostico MAPE

In [24]:
y_true = datos.loc[test.index, 'Demanda_SIN']
mae = mean_absolute_error(y_true, test['pronostico_xm'])
print("MAE:",mae)
mape = (mae / y_true).mean() * 100
print("MAPE:", mape) #MAPE DE XM los últimos 7 días se septiembre

MAE: 5461.915177428488
MAPE: 2.375653015635124


In [15]:
test

,RNN_forecast,LSTM_forecast,GRU_forecast,BidirectionalLSTM_forecast,Real Demand
FECHA,,,,,
2023-09-24,218153.4,215340.9,209295.3,213940.4,203949.3
2023-09-25,229240.9,228408.3,226251.5,227504.1,233453.1
2023-09-26,229858.1,229629.8,228741.5,228921.5,236506.3
2023-09-27,230113.5,229845.8,228569.5,229511.6,238964.9
2023-09-28,230560.0,230640.2,229329.4,230041.5,235940.4
2023-09-29,230779.4,230729.5,229204.0,229635.3,235929.7
2023-09-30,225600.0,225162.6,222462.7,224776.6,228839.6
